In [2]:
import requests
import json
import pandas as pd

In [3]:
NOTION_API_KEY = "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"
NOTION_URL = "https://api.notion.com/v1/pages"
PAGE_ID = "15b2ec8473e98007885fe7ec343ff6df"

headers = {
    "Authorization": "Bearer " + NOTION_API_KEY,
    "Content-Type": "application/json", 
    "Notion-Version": "2022-06-28", 
}

# Page ID : 15b2ec8473e98007885fe7ec343ff6df
# DB ID: 15b2ec8473e9800ebac5e2e551a7727d

In [28]:
#helper
def data_conversion(data: dict):
    response = {}
    for x in data:
        if data[x]["type"] == "title":
            tmp_data = {"title": [{"text": {"content": data[x]["value"]}}]}
        elif data[x]["type"] == "number":
            tmp_data = {"number": data[x]["value"]}
        elif data[x]["type"] == "rich_text":
            tmp_data = {"rich_text": [{"text": {"content": data[x]["value"]}}]}
        elif data[x]["type"] == "date":
            tmp_data = {"date": {"start": data[x]["value"], "end": None}}
        elif data[x]["type"] == "url":
            tmp_data = {"url": data[x]["value"]}

        response.update({x: tmp_data})
    
    return {"properties": response}

data = {
    "name" : {
        "type": "title",
        "value": "hoaihai"
    },
    "age" : {
        "type": "number",
        "value": 20
    }
}

output = data_conversion(data)
with open("output.json", "w") as f:
    json.dump(output, f, indent=4)

In [ ]:
#create
def create_customer(id:str, data: dict | None = None):
    pass

In [ ]:
#retrieve
def read_db():
    url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"
    response = requests.get(url, headers=headers)

    data = response.json()
    result = data["results"]

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.get(next_url, headers=headers)
        data = response.json()  
        result += data["results"]

    if result == []: 
        return None
    return result

def display_db():
    res = read_db()
    if res == None:
        return None

    for i in len(res):
        print(res[i]["child_database"]["title"])

def display_customer_data(dbID : str):
    url = f"https://api.notion.com/v1/databases/{dbID}/query"
    response = requests.post(url, headers=headers)

    data = response.json()
    result = data["results"]

    while data["has_more"]:
        next_url = url + f"?start_cursor={data['next_cursor']}"
        response = requests.post(next_url, headers=headers)
        data = response.json()  
        result += data["results"]
    
    if result == []:
        return None
    return result

def get_customer_data(customer_ID : str):
    res = read_db()
    if res == None:
        return None

    for i in len(res):
        if res[i]["child_database"]["title"] == customer_ID:
            return display_customer_data(res[i]["id"])
    
    return None

    
    

In [ ]:
#update
def update_customer_db(id : str, data: dict):
    url = f"https://api.notion.com/v1/pages/{id}"
    response = requests.patch(url, headers=headers, data=json.dumps(data))

    return response.json()

In [ ]:
#delete
def delete_customer_db(id : str):
    url = f"https://api.notion.com/v1/pages/{id}"
    response = requests.delete(url, headers=headers)

    return response.json()

In [ ]:
#test

In [24]:
url = f"https://api.notion.com/v1/blocks/{PAGE_ID}/children"

response = requests.get(url, headers=headers)

data  = response.json()

res = data["results"]

while data["has_more"]:
    next_url = f"{url}?start_cursor={data['next_cursor']}"
    response = requests.get(next_url, headers=headers)
    data = response.json()
    res.extend(data["results"])

dbid = res[0]["id"]

dburl = f"https://api.notion.com/v1/databases/{dbid}/query"

response2 = requests.post(dburl, headers=headers)

data2 = response2.json()

with open("data.json", "w") as f:
    json.dump(data2, f, indent=4)

res2 = data2["results"]

for x in res2[0]["properties"]: 
    print(res2[0]["properties"][x]["type"])

number
url
date
rich_text
title
